In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# === Imports ===
# Step 1: Imports
import sys, os, datetime, numpy as np
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Deep_Learning/mura_modules')

import datetime
import tensorflow as tf
from hybrid_model import build_hybrid_model, fine_tune_hybrid_model
from data_loader_hybrid import get_mura_dataset_hybrid
from config import BACKBONE_IMAGE_SIZES,BASE_PATH_GOOGLE_DRIVE, IMAGE_SIZE, BATCH_SIZE, NUM_CLASSES_BINARY
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import datetime
from data_loader_preprocessing import get_mura_dataset_hybrid
from metrics import get_binary_metrics
from plot_utils import plot_training_history

In [3]:
# === Settings ===
backbone = "InceptionV3"
image_shape = BACKBONE_IMAGE_SIZES[backbone] + (3,)
BATCH_SIZE = 32
EPOCHS = 5
EPOCHS_FINE_TUNE = 15

CSV_TRAIN = "/content/drive/MyDrive/Colab Notebooks/Deep_Learning/MURA-v1.1/train_image_metadata.csv"
CSV_VALID = "/content/drive/MyDrive/Colab Notebooks/Deep_Learning/MURA-v1.1/valid_image_metadata.csv"


In [4]:
import pandas as pd

df = pd.read_csv(CSV_TRAIN)
print(df.columns)


Index(['image_path', 'label', 'bodypart'], dtype='object')


In [5]:
# === Load datasets (hybrid) ===
train_ds = get_mura_dataset_hybrid(CSV_TRAIN, image_size=image_shape[:2], training=True, cache_dir="/content", backbone=backbone)
valid_ds = get_mura_dataset_hybrid(CSV_VALID, image_size=image_shape[:2], training=False, cache_dir="/content", backbone=backbone)


In [6]:
# === Build model ===
model, base_model = build_hybrid_model(
    image_shape=image_shape,
    num_categories=8,  # still valid
    backbone=backbone
)

model.summary()
base_model.summary()


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)  │ (None, 299, 299, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 149, 149, 32)   │            864 │ input_image[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 149, 149, 32)   │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 149, 149, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 147, 147, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 147, 147, 32)   │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 147, 147, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 147, 147, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 147, 147, 64)   │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 147, 147, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 73, 73, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 73, 73, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 73, 73, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 73, 73, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 71, 71, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 71, 71, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 22,066,337 (84.18 MB)

 Trainable params: 263,553 (1.01 MB)

...

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)  │ (None, 299, 299, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 149, 149, 32)   │            864 │ input_image[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 149, 149, 32)   │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 149, 149, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 147, 147, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 147, 147, 32)   │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 147, 147, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 147, 147, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 147, 147, 64)   │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 147, 147, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 73, 73, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 73, 73, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 73, 73, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 73, 73, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 71, 71, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 71, 71, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 21,802,784 (83.17 MB)

In [7]:
# === Callbacks ===
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M")

checkpoint_cb = ModelCheckpoint(
    filepath=f"/content/drive/MyDrive/Colab Notebooks/Deep_Learning/hybrid_inception_best_{timestamp}.h5",
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=False
)

earlystop_cb = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
lr_reduce_cb = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2)


In [8]:
# train_ds = train_ds.map(lambda img, part, label, weight: ((img, part), label, weight))
# valid_ds = valid_ds.map(lambda img, part, label, weight: ((img, part), label, weight))
#train_ds = train_ds.map(lambda inputs, label, weight: (inputs, label))
#valid_ds = valid_ds.map(lambda inputs, label, weight: (inputs, label))


TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.<lambda>() missing 1 required positional argument: 'weight'


In [9]:
# === Train hybrid model ===
history = model.fit(
    x=train_ds,
    validation_data=valid_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, earlystop_cb, lr_reduce_cb],
    verbose=1
)


Epoch 1/5
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8521 - loss: 0.3676 - precision: 0.8601 - recall: 0.8399

1151/1151 ━━━━━━━━━━━━━━━━━━━━ 2715s 2s/step - accuracy: 0.8521 - loss: 0.3676 - precision: 0.8600 - recall: 0.8399 - val_accuracy: 0.0431 - val_loss: 4.0271 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 2/5
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7839 - loss: 0.5775 - precision: 0.8160 - recall: 0.7732

1151/1151 ━━━━━━━━━━━━━━━━━━━━ 2204s 2s/step - accuracy: 0.7840 - loss: 0.5774 - precision: 0.8160 - recall: 0.7732 - val_accuracy: 0.0431 - val_loss: 3.9101 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 3/5
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7749 - loss: 0.5747 - precision: 0.8127 - recall: 0.7657

1151/1151 ━━━━━━━━━━━━━━━━━━━━ 2228s 2s/step - accuracy: 0.7750 - loss: 0.5746 - precision: 0.8127 - recall: 0.7657 - val_accuracy: 0.0431 - val_loss: 3.5168 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 4/5
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7642 - loss: 0.5938 - precision: 0.8219 - recall: 0.7337

1151/1151 ━━━━━━━━━━━━━━━━━━━━ 2194s 2s/step - accuracy: 0.7643 - loss: 0.5937 - precision: 0.8219 - recall: 0.7337 - val_accuracy: 0.0431 - val_loss: 2.9765 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 5/5
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7557 - loss: 0.5800 - precision: 0.8278 - recall: 0.7181

1151/1151 ━━━━━━━━━━━━━━━━━━━━ 2189s 2s/step - accuracy: 0.7557 - loss: 0.5799 - precision: 0.8278 - recall: 0.7181 - val_accuracy: 0.0431 - val_loss: 2.8750 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04


In [10]:
# === Save final model ===
model.save(f"/content/drive/MyDrive/Colab Notebooks/Deep_Learning/hybrid_inception_final_{timestamp}.h5")


In [ ]:
from plot_utils import plot_training_history
plot_training_history(history)

In [11]:
# Unfreeze and continue training
model = fine_tune_hybrid_model(model, model.get_layer(index=1), unfreeze_from=250)


AttributeError: 'Conv2D' object has no attribute 'layers'

In [ ]:
print(model.summary())

In [ ]:
history_fine = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=EPOCHS_FINE_TUNE,
    verbose=1
)

In [ ]:
from plot_utils import plot_training_history
plot_training_history(history_fine)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Extract predictions
y_true = []
y_pred = []

for (x, part), label in valid_ds.unbatch().take(1000):  # limit if needed
    pred = model.predict(([tf.expand_dims(x, 0), tf.expand_dims(part, 0)]), verbose=0)[0][0]
    y_pred.append(int(pred >= 0.5))
    y_true.append(int(label.numpy()))

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=["Normal", "Abnormal"]))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


In [ ]:
# === Save final model ===
model.save(f"/content/drive/MyDrive/Colab Notebooks/Deep_Learning/hybrid_inception_final_finetuned_{timestamp}.h5")
